In [ ]:
#Documentación: 
#https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_credentials_temp.html
#https://docs.aws.amazon.com/es_es/IAM/latest/UserGuide/id_roles_create_for-user.html
#https://docs.aws.amazon.com/es_es/STS/latest/APIReference/API_AssumeRole.html

In [4]:
import boto3
import pandas as pd
import matplotlib.pyplot as plt


# configuración de perfil
HUB_PROFILE   = "cloudwatch-report" 

#nombre del rol que se crea en las cuentas secuendarias
SAT_ROLE_NAME = "CloudWatchReadOnlyForHub"  

ACCOUNTS = [
    "209479294362",  
    "927750239354"
]
REGIONS = ["us-east-1"] 

ACCOUNT_NAMES = {
     "209479294362": "Cuenta Generation",
     "927750239354": "Cuenta Consejo Noruego"
}


hub_session = boto3.Session(profile_name=HUB_PROFILE)
sts_hub = hub_session.client("sts")

print("HUB identity:", sts_hub.get_caller_identity())

def assume_role(account_id: str, role_name: str, session_name: str = "jupyter-multi-test", duration_sec: int = 3600):
    role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
    try:
        resp = sts_hub.assume_role(
            RoleArn=role_arn,
            RoleSessionName=session_name,
            DurationSeconds=duration_sec
        )
        c = resp["Credentials"]
        return boto3.Session(
            aws_access_key_id=c["AccessKeyId"],
            aws_secret_access_key=c["SecretAccessKey"],
            aws_session_token=c["SessionToken"]
        )
    except ClientError as e:
        print(f"[{account_id}] ERROR AssumeRole: {e}")
        return None


HUB identity: {'UserId': 'AIDA2UC3DW7JCZSI7MSFV', 'Account': '730335524818', 'Arn': 'arn:aws:iam::730335524818:user/HubReporterRole', 'ResponseMetadata': {'RequestId': '6416ffed-31e3-4726-b497-aaf217e4865d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6416ffed-31e3-4726-b497-aaf217e4865d', 'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6MTc1OTQxNTAxNzc2NzpHOkEzVTd3cmVm', 'content-type': 'text/xml', 'content-length': '412', 'date': 'Thu, 02 Oct 2025 14:23:37 GMT'}, 'RetryAttempts': 0}}


In [8]:
from docx import Document
from docx.shared import Inches  

def listar_ec2(session: boto3.Session, region: str):
    ec2 = session.client("ec2", region_name=region)
    out = []
    paginator = ec2.get_paginator("describe_instances")
    try:
        for page in paginator.paginate():
            for res in page.get("Reservations", []):
                for inst in res.get("Instances", []):
                    name = next((t.get("Value") for t in inst.get("Tags", []) if t.get("Key") == "Name"), None)
                    lt = inst.get("LaunchTime")
                    if lt is not None:
                        if lt.tzinfo is None:
                            lt = lt.replace(tzinfo=timezone.utc)
                        lt = lt.isoformat()
                    out.append({
                        "Región": region,
                        "ID": inst["InstanceId"],
                        "Nombre": name,
                        "Estado": inst["State"]["Name"],
                        "Tipo": inst["InstanceType"],
                        "Zona": inst.get("Placement", {}).get("AvailabilityZone"),
                        "IP Privada": inst.get("PrivateIpAddress"),
                        "IP Pública": inst.get("PublicIpAddress"),
                        "Lanzamiento": lt,
                        "VpcId": inst.get("VpcId"),
                        "SubnetId": inst.get("SubnetId"),
                    })
        return out
    except ClientError as e:
        print(f"[{region}] ERROR describe_instances: {e}")
        return []

# Generar el docs por cuenta

for acct in ACCOUNTS:
    print(f"\n=== Cuenta: {acct} ===")
    friendly = ACCOUNT_NAMES.get(acct, acct)
    sat = assume_role(acct, SAT_ROLE_NAME)
    if sat is None:
        continue

    # 1) Recolección de datos por regiones
    rows = []
    for r in REGIONS:
        data = listar_ec2(sat, r)
        rows.extend(data)
        print(f"  {r}: {len(data)} instancia(s)")

    # 2) DataFrame y resumen
    df = pd.DataFrame(rows)
    if df.empty:
        # Aún así generamos un doc minimalista
        doc = Document()
        doc.add_heading(f"Reporte EC2 - Cuenta {friendly}", 0)
        doc.add_paragraph(
            f"No se encontraron instancias EC2 en las regiones configuradas ({', '.join(REGIONS)})."
        )
        out_doc = f"Reporte_EC2_{acct}.docx"
        doc.save(out_doc)
        print(f"✅ Informe vacío generado: {out_doc}")
        continue

    summary = (
        df.groupby(["Tipo", "Estado"])
          .size()
          .reset_index(name="Total")
          .sort_values(by="Total", ascending=False)
    )

    # 3) Gráfica por cuenta
    plt.figure(figsize=(10, 5))
    for state in summary["Estado"].unique():
        data = summary[summary["Estado"] == state]
        plt.bar(data["Tipo"], data["Total"], label=state)
    plt.title(f"Distribución de instancias EC2 - Cuenta {friendly}", fontsize=14)
    plt.ylabel("Cantidad de instancias")
    plt.xlabel("Tipo de instancia")
    plt.xticks(rotation=30)
    plt.legend()
    plt.tight_layout()
    chart_name = f"grafico_ec2_{acct}.png"
    plt.savefig(chart_name)
    plt.close()

    # 4) Documento Word por cuenta
    doc = Document()
    doc.add_heading(f"Reporte EC2 - Cuenta {friendly}", 0)
    doc.add_paragraph(
        f"Este informe resume las instancias EC2 de la cuenta {friendly}, "
        f"recopiladas mediante STS AssumeRole desde el perfil del hub '{HUB_PROFILE}'. "
        f"Regiones evaluadas: {', '.join(REGIONS)}."
    )

    # Detalle
    doc.add_heading("Instancias detectadas", level=1)
    detail_cols = ["Región","ID","Nombre","Estado","Tipo","Zona","IP Privada","IP Pública","Lanzamiento","VpcId","SubnetId"]
    t1 = doc.add_table(rows=1, cols=len(detail_cols))
    hdr = t1.rows[0].cells
    for i, col in enumerate(detail_cols):
        hdr[i].text = col
    for _, row in df[detail_cols].iterrows():
        c = t1.add_row().cells
        for i, val in enumerate(row):
            c[i].text = "" if pd.isna(val) else str(val)

    # Resumen
    doc.add_heading("Resumen agrupado (Tipo x Estado)", level=1)
    res_cols = list(summary.columns)
    t2 = doc.add_table(rows=1, cols=len(res_cols))
    hdr2 = t2.rows[0].cells
    for i, col in enumerate(res_cols):
        hdr2[i].text = col
    for _, row in summary.iterrows():
        c = t2.add_row().cells
        for i, val in enumerate(row):
            c[i].text = str(val)

    # Gráfica
    doc.add_heading("Visualización", level=1)
    doc.add_picture(chart_name, width=Inches(6))

    out_doc = f"Reporte_EC2_{acct}.docx"
    doc.save(out_doc)

    print(f"✅ Informe generado: {out_doc}")
    print(f"📊 Gráfica: {chart_name}")
    print(f"📦 Total instancias: {len(df)}")


=== Cuenta: 209479294362 ===
  us-east-1: 1 instancia(s)
✅ Informe generado: Reporte_EC2_209479294362.docx
📊 Gráfica: grafico_ec2_209479294362.png
📦 Total instancias: 1

=== Cuenta: 927750239354 ===
  us-east-1: 3 instancia(s)
✅ Informe generado: Reporte_EC2_927750239354.docx
📊 Gráfica: grafico_ec2_927750239354.png
📦 Total instancias: 3
